In [1]:
#import torchnet as tnt
import torch
print(torch.__version__)

1.4.0


In [2]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = 'saeid-cat-dog'

outputpath='s3://saeid-cat-dog/output'
training_path = 's3://saeid-cat-dog/data'

role = sagemaker.get_execution_role()

## Script

Show the entry point script inline for reference

In [12]:
!pygmentize train.py

from __future__ import print_function, division
import sagemaker_containers
import logging
import sys
import PIL
import argparse
import json
import os
import torch
import numpy as np
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torch.utils.data.distributed
from torch.optim import lr_scheduler
from torchvision import datasets, transforms, models
from collections import OrderedDict
import boto3
import tarfile
import datetime
import math
import copy
import torchnet as tnt


logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))
validation_loss=[]
min_val_loss=9999
train_accuracy=[]
validation_accuracy=[]
#MEAN = [0.413, 0.413, 0.413]
#STD = [0.159, 0.161, 0.158]

MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]


def _average_gradients(model):
    # Gradient averaging.
    size = float(dist.get_world_size())
    for 

    # recommended way from http://pytorch.org/docs/master/notes/serialization.html
    #torch.save(model.cpu().state_dict(), path)
    #torch.save(model.cpu().module.state_dict(), path)

    # save full model - model.module will be the method to predict image
    #torch.save(model, path)
    # save the part inside parallel data 
    torch.save(model.cpu().module, path)

#
#------------------------------------ Data Loader ---------------------------------------------
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))

MEAN = [0.413, 0.413, 0.413]
STD = [0.159, 0.161, 0.158]


class Make_weights_for_balanced_classes():
    
    def __init__(self,images,nclasses):
        self.images = images
        self.nclasses = nclasses

    def get_weights(self):
        count = [0] * self.nclasses                                                      
        for item in self.images:                                                

## Training

### Create PyTorch estimator

Instantiate a sagemaker estimator object and pass it the entry point script and path to save the model

This is also where you may define non-default hyperparameters found in the entry point script

In [5]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point="train.py",
                    output_path=outputpath,
                    base_job_name='resnet50-cat-dog',
                    train_volume_size=20,
                    role=role,
                    framework_version='1.0', 
                    train_instance_count=1,
                    train_max_run=(5*24*60*60),
                    train_instance_type='ml.p3.2xlarge',
                    #train_instance_type='ml.p2.8xlarge',
                    hyperparameters={
                        'batch_size': 64,
                        'lr': 0.01,
                        'lr_step': 10,
                        'data_location':training_path,
                        'epochs':5,
                        'mean': [0.485, 0.456, 0.406],
                        'std':[0.229, 0.224, 0.225],
                        'no_of_classes':2,             
                        'model_def':'resnet50'
                    }) # hyperparameters gets replaced with a config file using SGD with warm start.

In [6]:
estimator.fit({'training': training_path})

2020-04-16 08:39:15 Starting - Starting the training job...
2020-04-16 08:39:17 Starting - Launching requested ML instances......
2020-04-16 08:40:16 Starting - Preparing the instances for training......
2020-04-16 08:41:18 Downloading - Downloading input data......
2020-04-16 08:42:34 Training - Downloading the training image..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-04-16 08:42:56,145 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-04-16 08:42:56,172 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-04-16 08:42:57,586 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-04-16 08:42:57,797 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2020-04-16 08:42:57,797 sagemaker-containers INFO     Generating setup.cfg
2020-04-16 08:42:57,797 sagemaker-container


2020-04-16 08:42:54 Training - Training image download completed. Training in progress.Model on cuda: True
Model on cuda: True
Epoch 1/5
Epoch 1/5
----------
----------
Train accuracy: [tensor(0.9352, device='cuda:0', dtype=torch.float64)]
Train accuracy: [tensor(0.9352, device='cuda:0', dtype=torch.float64)]
Train Loss: 0.1606
Train Loss: 0.1606
Train Acc: 0.9352
Train Acc: 0.9352
Validation accuracy: [tensor(0.9930, device='cuda:0', dtype=torch.float64)]
Validation accuracy: [tensor(0.9930, device='cuda:0', dtype=torch.float64)]
Val Loss: 0.0260
Val Loss: 0.0260
Val Acc: 0.9930
Val Acc: 0.9930


Learning rate: 0.0021
Learning rate: 0.0021
storing best model Loss: 0.0260
storing best model Loss: 0.0260
Epoch 2/5
Epoch 2/5
----------
----------
Train accuracy: [tensor(0.9352, device='cuda:0', dtype=torch.float64), tensor(0.9828, device='cuda:0', dtype=torch.float64)]
Train accuracy: [tensor(0.9352, device='cuda:0', dtype=torch.float64), tensor(0.9828, device='cuda:0', dtype=torch.floa